### Environment Setup

Imports

In [1]:
## This cell will install dask If not installed yet.
# If so please run this notebook again as soon as this cell
# finishes executing. 

try:
  import dask.dataframe as dd   
except ImportError:
  !pip install "dask[complete]" --upgrade
  print("dask installed\n\nPlease RUN this notebook again!")

  # Restarting runtime
  import os
  os.kill(os.getpid(), 9)

#----

#import dask.array as da
from dask.distributed import Client
from zipfile import ZipFile
import os, shutil

Montando o Google Drive

In [2]:
## ATENÇÃO! Aparecerá um link externo seguro provido automaticamente pelo Google Colab para que seja 
## gerado o código de autenticação necessário para o acesso ao Drive.

if not os.path.exists('/content/drive'):
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


Setup dask client

In [3]:
# Dask client setup (Colab RAM)
client = Client()
client

Client Scheduler: tcp://127.0.0.1:44025 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 12.72 GiB


Criando pasta para armazenamento dos dados

In [4]:
if not os.path.exists('/content/DADOS/'):
  !mkdir '/content/DADOS/'

### Data Gathering

Unzipping data folder

In [5]:
def unzip_csv(ano):
  if not isinstance(ano, str):
    ano = str(ano)
  
  zipdata_path = f'/content/drive/MyDrive/Colab Notebooks/IC_Data_Science_ENEM/Dados/microdados_enem_{ano}.zip'
  target_path  = f'/content/DADOS/MICRODADOS_ENEM_{ano}.csv'
  source_path  = f'DADOS/MICRODADOS_ENEM_{ano}.csv'

  if os.path.exists(target_path):
    return True

  # Os dados do ano de 2017 foram compactados de forma diferente:
  if ano == '2017':
    source_path = f'Microdados Enem 2017/{source_path}'

  # Copiando apenas o arquivo .csv para a pasta destino sem todo o caminho
  # herdado do diretório
  with ZipFile(zipdata_path, 'r') as z:
    source = z.open(source_path)
    target = open(target_path, 'wb')

    with source, target:
      shutil.copyfileobj(source, target)

In [6]:
ano = 2018
unzip_csv(ano)

Reading data into dask dataframe

In [7]:
data_path      = f'/content/DADOS/MICRODADOS_ENEM_{str(ano)}.csv'
temp_save_path = f'/content/DADOS/MICRODADOS_ENEM_{str(ano)}.pq'

In [8]:
%%time
if not os.path.exists(temp_save_path):
  # read from csv file
  df = dd.read_csv(data_path, sep=';', encoding='latin-1',
                   dtype={'NU_IDADE':       'float64',
                          'TP_PRESENCA_CH': 'float64',
                          'TP_PRESENCA_CN': 'float64',
                          'TP_PRESENCA_LC': 'float64',
                          'TP_PRESENCA_MT': 'float64'})
  # converto to parquet file
  df.to_parquet(temp_save_path)

# read parquet file (more efficient)
df = dd.read_parquet(temp_save_path)
print(df.head())

   NU_INSCRICAO  NU_ANO  CO_MUNICIPIO_RESIDENCIA  ... Q025  Q026 Q027
0  180008202043    2018                  5300108  ...    A     B    A
1  180007197856    2018                  2111102  ...    A     A    A
2  180008517434    2018                  3530607  ...    B     A    A
3  180007661228    2018                  2916401  ...    B     A    A
4  180008787987    2018                  2918100  ...    B     B    A

[5 rows x 137 columns]
CPU times: user 8.74 s, sys: 1.09 s, total: 9.83 s
Wall time: 2min 11s


Informações sobre o dataframe

In [9]:
%%time
# Dataframe dimensions
print(f'Rows: {len(df):,}\nColumns: {df.shape[1]}\n')

Rows: 5,513,747
Columns: 137

CPU times: user 252 ms, sys: 32.9 ms, total: 285 ms
Wall time: 909 ms


In [10]:
%%time
print(f'Número de partições: {df.npartitions}')

Número de partições: 54
CPU times: user 1.63 ms, sys: 0 ns, total: 1.63 ms
Wall time: 1.53 ms


In [11]:
%%time
# Partition Average Sizes 
df.map_partitions(len).compute().head()   # Takes longer than other cells due to compute() method

CPU times: user 2.36 s, sys: 257 ms, total: 2.62 s
Wall time: 30.1 s


0    101586
1    101274
2    100626
3     99743
4     99695
dtype: int64

In [12]:
%%time
# Accessing partition 1 (second one)
df.get_partition(1).head()

CPU times: user 59.5 ms, sys: 4.68 ms, total: 64.2 ms
Wall time: 735 ms


,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_MUNICIPIO_NASCIMENTO,NO_MUNICIPIO_NASCIMENTO,CO_UF_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_ESCOLA,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_SURDEZ,IN_DEFICIENCIA_AUDITIVA,IN_SURDO_CEGUEIRA,IN_DEFICIENCIA_FISICA,IN_DEFICIENCIA_MENTAL,IN_DEFICIT_ATENCAO,IN_DISLEXIA,IN_DISCALCULIA,IN_AUTISMO,IN_VISAO_MONOCULAR,...,TX_RESPOSTAS_MT,TP_LINGUA,TX_GABARITO_CN,TX_GABARITO_CH,TX_GABARITO_LC,TX_GABARITO_MT,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q026,Q027
0,180009384900,2018,3146107,Ouro Preto,31,MG,32.0,F,0.0,3,1,3304557.0,Rio de Janeiro,33.0,RJ,1,12,1,1.0,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,None,0,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H,H,F,F,5,G,A,B,C,A,A,B,A,B,A,B,A,A,C,A,A,D,A,B,B,A,A
1,180007396048,2018,2304400,Fortaleza,23,CE,43.0,M,0.0,3,1,2304400.0,Fortaleza,23.0,CE,1,12,1,NaN,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,DBDDEECDCBCCBDCBCCAAADBDEAEECECCCDDACCECDAADD,0,BDDEDBCACEBCCACDCDDAECAADBDBCEEAEAABEBEBBACED,ECCBBEBEDBEDBDDAEECDECBECACDEDEECDECAABBDEBCE,BCDEBDBCBBACDCBEAECBABADBBECECBDBBEABDADACEDEC...,ACEADCEBBDADAAEBBDDEDABADBEXCCCDBCAEEACBCDDAB,1.0,100.0,120.0,120.0,120.0,120.0,580.0,D,A,F,F,3,D,B,A,A,E,A,A,A,A,A,B,A,A,A,A,A,A,B,B,B,A,B
2,180009169532,2018,3146107,Ouro Preto,31,MG,22.0,F,0.0,3,2,NaN,None,NaN,None,1,5,1,1.0,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,BACBCDBDEDCACCEACBDDBDBEEDCADCAECBEEBECABEBCB,0,BDDEDBCACEBCCACDCDDAECAADBDBCEEAEAABEBEBBACED,AEECDECBEBDCACDEDEEDBDDEBCECDECAABEDBEBBEBECC,BBEDCCBBDBBBEADECACEDBEADABEDBABAECCBEABDDBACD...,ACEADCEBBDADAAEBBDDEDABADBEXCCCDBCAEEACBCDDAB,1.0,140.0,160.0,140.0,140.0,120.0,700.0,E,E,B,B,5,G,A,C,D,B,A,B,A,B,A,B,A,A,C,A,A,E,A,B,A,A,A
3,180009235449,2018,4314902,Porto Alegre,43,RS,27.0,M,NaN,3,1,4314902.0,Porto Alegre,43.0,RS,2,0,2,1.0,0,43108318.0,4314902.0,Porto Alegre,43.0,RS,2.0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,...,ACDDBABECAADCAABDCDABBBADBDBCEECBDDBCADCBBEBD,1,BDDEDBCACEBCCACDCDDAECAADBDBCEEAEAABEBEBBACED,ECCBBEBEDBEDBDDAEECDECBECACDEDEECDECAABBDEBCE,BCDEBDBCBBACDCBEAECBABADBBECECBDBBEABDADACEDEC...,ACEADCEBBDADAAEBBDDEDABADBEXCCCDBCAEEACBCDDAB,1.0,120.0,120.0,120.0,80.0,80.0,520.0,B,B,C,C,4,E,A,B,B,A,A,B,B,B,A,B,A,A,C,B,B,D,A,B,B,B,A
4,180008147022,2018,2305803,Ipu,23,CE,40.0,F,1.0,3,1,2305803.0,Ipu,23.0,CE,2,0,2,3.0,0,23231939.0,2305803.0,Ipu,23.0,CE,2.0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,...,None,1,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,B,A,A,5,B,A,A,A,A,A,B,A,A,A,A,A,A,B,A,A,B,A,A,A,B,A


Com *dask*, sempre que precisar utilizar alguma funcionalidade suportada apenas por *Pandas*, usa-se *compute()* - ou *len(dask_df)*, como acima - que "varre" todas as partições em que os dados foram armazenados. O tempo decorrido é inversamente proporcional ao espaço em memória RAM e ao número de "trabalhadores" utilizados (por padrão, igual ao número de núcleos presente no processador).

No Google Colab:

1.   Memória RAM alocável: **12,72 GB**

2.   Número de núcleos (*workers*) disponível: **2 núcleos**

Algumas operações podem tomar um certo tempo devido ao número relativamente baixo de núcleos no amiente virtual. Entretanto, é apreciado que seja possível ter todos os dados sem que a memória RAM seja sobrecarregada ou que sejam excluídas colunas antes mesmo de serem analisadas.

Para isso, tem-se 50 partições com aproximadamente 100 mil linhas de dados.



### Pré-processamento de Dados

*Roadmap*:

Manipulando colunas

*   TP_PRESENCA: 0 if any 0 in TP_PRESENCAs else 1  #ok
*   NU_NOTA_MEDIA: média das NU_NOTAs               #ok
*   Atendimento Especializado/Específico            #ok

Excluindo colunas

*   Colunas sobre escola com proporção de valores nulos muito alta (>50%?) #ok
*   Respostas, gabaritos, notas de competências e códigos de provas       #ok
*   Pertencentes a IN_SEM_RECURSO     #ok



#### Manipulando colunas

Agrupando dados espalhados em múltiplas colunas no menor número possível de counas a fim de reduzir as dimensões da base de dados.

**Coluna de Média de Notas**

Para dar simplicidade e objetividade ao algoritmo posteriormente, é interessante utilizar apenas uma colunas como alvo com relação às notas. Portanto, as notas serão representadas pela média de cada uma das 5 (cinco) áeras do conhecimento.

Além disso, todas as demais colunas relacionadas às notas (notas de competência na Redação, nota em cada cadernos de prova, folhas-resposta e gabaritos)

In [13]:
cols_notas = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']
df = df.assign(NU_NOTA_MEDIA = df[cols_notas].mean(axis=1, skipna=False))   
# Se quiser saber somente quantos não foram em nenhum dia, basta tirar o parâmetro skipna

**Coluna de Presença**

Se o inscrito constar ausência em algum ou em todos os dias será 
registrado como *TP_PRESENCA: False*. Dessa forma, a definição de ausente/desistente é obtida com maior clareza a partir de uma única coluna. Aquelas que serviram de base para se obter TP_PRESENCA serão excluídas.

In [14]:
# Forma vetorizada - desempenho maior
#df['TP_PRESENCA'] = (
#    (df.TP_PRESENCA_CH==1) & (df.TP_PRESENCA_CN==1) & (df.TP_PRESENCA_LC==1) & (df.TP_PRESENCA_MT==1) & (df.TP_STATUS_REDACAO==1)
#)

df['TP_PRESENCA'] = ~df['NU_NOTA_MEDIA'].isna()
df['TP_PRESENCA'] = df['TP_PRESENCA'].astype(int)

In [15]:
%%time
df['TP_PRESENCA'].value_counts().compute()  # o parâmetro normalize=True não tem suporte para Dask

CPU times: user 2.32 s, sys: 249 ms, total: 2.57 s
Wall time: 32.6 s


1    3893743
0    1620004
Name: TP_PRESENCA, dtype: int64

Em *pandas*, o método *value_counts* possui um parâmetro chamado *normalize* que retorna a frequência de cada valor. Em *Dask*, entretanto, não é suportado.

Sendo assim, realizando um cálculo manual. tem-se que a **razão de desistência ou ausência, em qualquer das cinco provas, é de 27,34%** 

**Coluna de Atendimento Especializado**

O INEP oferece opções de atendimento especializado que são dispostas em 13 colunas. Como compreendem uma pequena parte da composição, serão agrupadas em uma só coluna que indicará se o inscrito requisitou ou não algum atendimento especializado para o Exame.

In [16]:
cols_especial = ['IN_BAIXA_VISAO', 'IN_CEGUEIRA', 'IN_SURDEZ', 'IN_DEFICIENCIA_AUDITIVA',
                 'IN_SURDO_CEGUEIRA', 'IN_DEFICIENCIA_FISICA', 'IN_DEFICIENCIA_MENTAL',
                 'IN_DEFICIT_ATENCAO', 'IN_DISLEXIA', 'IN_DISCALCULIA', 'IN_AUTISMO',
                 'IN_VISAO_MONOCULAR', 'IN_OUTRA_DEF']

df = df.assign(IN_ATEND_ESPECIALIZADO = df.loc[:, cols_especial].max(1))

**Coluna de Atendimento Específico**

O Inep oferece opções de atendimento a condições específicas que são dispostas em 4 colunas. Como compreendem uma pequena parte da composição, serão agrupadas em uma só coluna que indicará se o inscrito requisitou ou não algum atendimento específico para o Exame.

In [17]:
cols_especifico = ['IN_GESTANTE', 'IN_LACTANTE', 'IN_IDOSO', 'IN_ESTUDA_CLASSE_HOSPITALAR']
df = df.assign(IN_ATEND_ESPECIFICO = df.loc[:, cols_especifico].max(1))

In [18]:
df.head()

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_MUNICIPIO_NASCIMENTO,NO_MUNICIPIO_NASCIMENTO,CO_UF_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_ESCOLA,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_SURDEZ,IN_DEFICIENCIA_AUDITIVA,IN_SURDO_CEGUEIRA,IN_DEFICIENCIA_FISICA,IN_DEFICIENCIA_MENTAL,IN_DEFICIT_ATENCAO,IN_DISLEXIA,IN_DISCALCULIA,IN_AUTISMO,IN_VISAO_MONOCULAR,...,TX_GABARITO_LC,TX_GABARITO_MT,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q026,Q027,NU_NOTA_MEDIA,TP_PRESENCA,IN_ATEND_ESPECIALIZADO,IN_ATEND_ESPECIFICO
0,180008202043,2018,5300108,Brasília,53,DF,44.0,M,1.0,1,0,NaN,None,NaN,None,2,0,2,1.0,0,52033333.0,5208707.0,Goiânia,52.0,GO,2.0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,A,A,A,1,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,NaN,0,0,0
1,180007197856,2018,2111102,São João dos Patos,21,MA,23.0,F,0.0,3,1,2111102.0,São João dos Patos,21.0,MA,1,5,1,1.0,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,BBDECBBCDBCECBABDBEDACDDBBBEADECACEDBEBABCBEAE...,ADCACEDDABEBBCBCADDACCCDEBBDAEEDBEXBCAEDABADA,1.0,120.0,40.0,40.0,60.0,20.0,280.0,B,B,A,A,4,B,A,B,C,A,A,B,A,A,A,A,A,A,B,A,A,B,A,A,A,A,A,472.62,1,0,0
2,180008517434,2018,3530607,Mogi das Cruzes,35,SP,23.0,F,0.0,1,1,4209300.0,Lages,42.0,SC,1,3,1,3.0,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E,E,C,B,4,C,A,B,C,B,A,B,A,B,A,B,A,A,B,A,A,B,B,B,B,A,A,NaN,0,0,0
3,180007661228,2018,2916401,Itapetinga,29,BA,26.0,F,0.0,3,2,NaN,None,NaN,None,1,8,1,1.0,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,C,C,B,4,B,A,B,C,A,A,B,A,A,A,B,A,A,B,B,A,C,A,B,B,A,A,NaN,0,0,0
4,180008787987,2018,2918100,Jeremoabo,29,BA,20.0,M,0.0,0,1,2800308.0,Aracaju,28.0,SE,2,0,2,1.0,0,29112699.0,2918100.0,Jeremoabo,29.0,BA,2.0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,...,CBEDBBBDCBECADBBBECBEAECABDADACDBABCECBBEDADAB...,None,1.0,140.0,40.0,40.0,120.0,20.0,360.0,C,E,A,D,3,B,A,B,C,A,B,B,A,A,A,A,A,A,B,A,A,B,A,B,B,B,A,NaN,0,0,0


#### Excluindo colunas

**Colunas de Presenças, Notas, Atendimento Especializado e Antendimento Específico**

Após ter reunidas as informações de tantas colunas em um número bem menor, pode-se, agora, excluír essas colunas sem que se perca informação relevante e, assim, reduzir as dimensões da base de dados.

In [19]:
cols_presenca = ['TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT', 'TP_STATUS_REDACAO']
cols_notas    = ['CO_PROVA_CN', 'TP_LINGUA', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2',
                 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_LC',
                 'TX_RESPOSTAS_MT', 'TX_GABARITO_CN', 'TX_GABARITO_CH', 'TX_GABARITO_LC', 'TX_GABARITO_MT']
cols_to_delete = cols_presenca + cols_notas + cols_especial + cols_especifico
#print(cols_to_delete)

df = df.drop(cols_to_delete, axis=1)

In [20]:
# Por alguma razão, é necessário excluir estas colunas separadamente
# Se excluir junto com as demais, os valores da média de notas se alteram       ## Checar motivo
df = df.drop(['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT'], axis=1)

In [21]:
df.head()

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_MUNICIPIO_NASCIMENTO,NO_MUNICIPIO_NASCIMENTO,CO_UF_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_ESCOLA,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,IN_SEM_RECURSO,IN_BRAILLE,IN_AMPLIADA_24,IN_AMPLIADA_18,IN_LEDOR,IN_ACESSO,IN_TRANSCRICAO,IN_LIBRAS,IN_LEITURA_LABIAL,IN_MESA_CADEIRA_RODAS,IN_MESA_CADEIRA_SEPARADA,IN_APOIO_PERNA,...,IN_SALA_ACOMPANHANTE,IN_MOBILIARIO_ESPECIFICO,IN_MATERIAL_ESPECIFICO,IN_NOME_SOCIAL,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q026,Q027,NU_NOTA_MEDIA,TP_PRESENCA,IN_ATEND_ESPECIALIZADO,IN_ATEND_ESPECIFICO
0,180008202043,2018,5300108,Brasília,53,DF,44.0,M,1.0,1,0,NaN,None,NaN,None,2,0,2,1.0,0,52033333.0,5208707.0,Goiânia,52.0,GO,2.0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,5210208,Iporá,52,GO,NaN,A,A,A,A,1,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,NaN,0,0,0
1,180007197856,2018,2111102,São João dos Patos,21,MA,23.0,F,0.0,3,1,2111102.0,São João dos Patos,21.0,MA,1,5,1,1.0,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2111102,São João dos Patos,21,MA,280.0,B,B,A,A,4,B,A,B,C,A,A,B,A,A,A,A,A,A,B,A,A,B,A,A,A,A,A,472.62,1,0,0
2,180008517434,2018,3530607,Mogi das Cruzes,35,SP,23.0,F,0.0,1,1,4209300.0,Lages,42.0,SC,1,3,1,3.0,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,4314902,Porto Alegre,43,RS,NaN,E,E,C,B,4,C,A,B,C,B,A,B,A,B,A,B,A,A,B,A,A,B,B,B,B,A,A,NaN,0,0,0
3,180007661228,2018,2916401,Itapetinga,29,BA,26.0,F,0.0,3,2,NaN,None,NaN,None,1,8,1,1.0,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2916401,Itapetinga,29,BA,NaN,B,C,C,B,4,B,A,B,C,A,A,B,A,A,A,B,A,A,B,B,A,C,A,B,B,A,A,NaN,0,0,0
4,180008787987,2018,2918100,Jeremoabo,29,BA,20.0,M,0.0,0,1,2800308.0,Aracaju,28.0,SE,2,0,2,1.0,0,29112699.0,2918100.0,Jeremoabo,29.0,BA,2.0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2918100,Jeremoabo,29,BA,360.0,C,E,A,D,3,B,A,B,C,A,B,B,A,A,A,A,A,A,B,A,A,B,A,B,B,B,A,NaN,0,0,0


**Colunas de Recursos Especiais e Específicos**

O Inep oferece, ainda, a opção do inscrito solicitar algum recurso especializado ou específico, como prova em Braille, protetor auricular, intérpretes, leito para realizar prova deitado, entre outros.

Existe, porém, na base de dados fornecida pelo MEC/Inep, a coluna *IN_SEM_RECURSO* que já informa se o inscrito solicitou ou não algum desses recursos e, portanto, não foi necessário reunir os dados de todas as colunas relacionadas a esse indicador para uma única coluna como foi feito acima com os outros indicadores.

Desta forma, entende-se por necessário manter apensas a coluna *IN_SEM_RECURSO* e excluir as outras, visto que nela já estão contidas todas as informações relevantes para o estudo.

In [22]:
cols_recurso = ['IN_BRAILLE', 'IN_AMPLIADA_24', 'IN_AMPLIADA_18', 'IN_LEDOR', 'IN_ACESSO',
                'IN_TRANSCRICAO', 'IN_LIBRAS', 'IN_TEMPO_ADICIONAL', 'IN_LEITURA_LABIAL',
                'IN_MESA_CADEIRA_RODAS', 'IN_MESA_CADEIRA_SEPARADA', 'IN_APOIO_PERNA',
                'IN_GUIA_INTERPRETE', 'IN_COMPUTADOR', 'IN_CADEIRA_ESPECIAL',
                'IN_CADEIRA_CANHOTO', 'IN_CADEIRA_ACOLCHOADA', 'IN_PROVA_DEITADO',
                'IN_MOBILIARIO_OBESO', 'IN_LAMINA_OVERLAY', 'IN_PROTETOR_AURICULAR',
                'IN_MEDIDOR_GLICOSE', 'IN_MAQUINA_BRAILE', 'IN_SOROBAN', 'IN_MARCA_PASSO',
                'IN_SONDA', 'IN_MEDICAMENTOS', 'IN_SALA_INDIVIDUAL', 'IN_SALA_ESPECIAL',
                'IN_SALA_ACOMPANHANTE', 'IN_MOBILIARIO_ESPECIFICO', 'IN_MATERIAL_ESPECIFICO',
                'IN_NOME_SOCIAL']

df = df.drop(cols_recurso, axis=1, errors='ignore')

In [23]:
df.head()   # Já é possível ver todas as colunas do dataframe sem cortes/pulos

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_MUNICIPIO_NASCIMENTO,NO_MUNICIPIO_NASCIMENTO,CO_UF_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_ESCOLA,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,IN_SEM_RECURSO,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q026,Q027,NU_NOTA_MEDIA,TP_PRESENCA,IN_ATEND_ESPECIALIZADO,IN_ATEND_ESPECIFICO
0,180008202043,2018,5300108,Brasília,53,DF,44.0,M,1.0,1,0,NaN,None,NaN,None,2,0,2,1.0,0,52033333.0,5208707.0,Goiânia,52.0,GO,2.0,1.0,1.0,0,5210208,Iporá,52,GO,NaN,A,A,A,A,1,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,NaN,0,0,0
1,180007197856,2018,2111102,São João dos Patos,21,MA,23.0,F,0.0,3,1,2111102.0,São João dos Patos,21.0,MA,1,5,1,1.0,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,2111102,São João dos Patos,21,MA,280.0,B,B,A,A,4,B,A,B,C,A,A,B,A,A,A,A,A,A,B,A,A,B,A,A,A,A,A,472.62,1,0,0
2,180008517434,2018,3530607,Mogi das Cruzes,35,SP,23.0,F,0.0,1,1,4209300.0,Lages,42.0,SC,1,3,1,3.0,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,4314902,Porto Alegre,43,RS,NaN,E,E,C,B,4,C,A,B,C,B,A,B,A,B,A,B,A,A,B,A,A,B,B,B,B,A,A,NaN,0,0,0
3,180007661228,2018,2916401,Itapetinga,29,BA,26.0,F,0.0,3,2,NaN,None,NaN,None,1,8,1,1.0,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,2916401,Itapetinga,29,BA,NaN,B,C,C,B,4,B,A,B,C,A,A,B,A,A,A,B,A,A,B,B,A,C,A,B,B,A,A,NaN,0,0,0
4,180008787987,2018,2918100,Jeremoabo,29,BA,20.0,M,0.0,0,1,2800308.0,Aracaju,28.0,SE,2,0,2,1.0,0,29112699.0,2918100.0,Jeremoabo,29.0,BA,2.0,1.0,1.0,0,2918100,Jeremoabo,29,BA,360.0,C,E,A,D,3,B,A,B,C,A,B,B,A,A,A,A,A,A,B,A,A,B,A,B,B,B,A,NaN,0,0,0


**Verificando a frequência de valores nulos**

In [24]:
%%time
# Might take a little long - 2x compute() and 1x len(dask_df)
df_columns_with_nan = df[df.columns[df.isna().any().compute()]]  # Parte do dataframe original apenas com as colunas que contem valores nulos
df_nan = df_columns_with_nan.isna().sum().compute().to_frame()   # Dataframe com a soma de valores nulos de cada coluna de df_withnan

df_nan.rename(columns={0: 'Quantidade'}, inplace=True)
df_nan['Porcentagem'] = 100*df_nan['Quantidade']/len(df_columns_with_nan)

CPU times: user 10.4 s, sys: 1.11 s, total: 11.5 s
Wall time: 2min 25s


In [25]:
df_nan

,Quantidade,Porcentagem
NU_IDADE,97,0.001759
TP_ESTADO_CIVIL,217637,3.947171
CO_MUNICIPIO_NASCIMENTO,185783,3.369451
NO_MUNICIPIO_NASCIMENTO,185783,3.369451
CO_UF_NASCIMENTO,185783,3.369451
SG_UF_NASCIMENTO,185783,3.369451
TP_ENSINO,2030666,36.829147
CO_ESCOLA,4064926,73.723477
CO_MUNICIPIO_ESC,4064926,73.723477
NO_MUNICIPIO_ESC,4064926,73.723477


Decidiu-se por excluir aqueles que possuírem valores nulos em NU_IDADE, TP_ESTADO_CIVIL e Q026

In [26]:
subset = ['NU_IDADE', 'TP_ESTADO_CIVIL', 'Q026']
df = df.dropna(how='any', subset=subset)

Porcentagem de dados faltantes a respeito das escolas em cada estado brasileiro

In [27]:
import pandas as pd
import plotly.express as px

In [28]:
%%time
esc_uf = df.groupby('SG_UF_PROVA')['SG_UF_ESC']
esc_uf.count().head(27)

CPU times: user 3.45 s, sys: 388 ms, total: 3.84 s
Wall time: 48.3 s


In [29]:
%%time
ausencia_esc_uf_pct = pd.crosstab(
    df['SG_UF_PROVA'],df['SG_UF_ESC'].fillna('NaN'), colnames=['UF_ESC'], rownames=['UF_PROVA']
)

CPU times: user 24.4 s, sys: 3.17 s, total: 27.6 s
Wall time: 4min 52s


In [30]:
ausencia_esc_uf_pct  = ausencia_esc_uf_pct.apply(lambda x: x/x.sum()*100, axis=1)

stacked_esc = ausencia_esc_uf_pct.reset_index().rename(columns={'NaN':'value'})
stacked_esc = stacked_esc.sort_values(by='value', ascending=False)

In [32]:
fig = px.bar(
    stacked_esc, x='UF_PROVA', y='value',
    text = 'value',
    labels={
        'value': 'Dados Ausentes [%]',
        'UF_PROVA': 'Estado'},
    height=600, width=1200,
    title=f'Percentual de Dados Ausentes sobre as Escolas por Estado do Brasil no ENEM {str(ano)}'
    )
#fig.update_layout(
#    font_family="Courier New",
#    font_color="blue",
#    title_font_family="Times New Roman",
#    title_font_color="red"
#)
fig.update_layout(
    title={
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}    
)
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.show()

É possível observar que diversas informações sobre as escolas estão com uma taxa de omissão bem alta (aproximadamente 73% dos dados totais estão faltantes).

Como não há uma notável diferença de proporção entre os estados, não é possível inferir nenhuma relação tampouco tomar qualquer decisão baseada nesses dados, já que fica muito difícil de substituí-los ou deletar os inscritos sem esses dados.

In [33]:
columns_to_drop = ['CO_ESCOLA', 'CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC', 'CO_UF_ESC', 'SG_UF_ESC',
                   'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC', 'TP_ENSINO']
df = df.drop(labels=columns_to_drop, axis=1, errors='ignore')

In [34]:
df.head()

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_MUNICIPIO_NASCIMENTO,NO_MUNICIPIO_NASCIMENTO,CO_UF_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,IN_TREINEIRO,IN_SEM_RECURSO,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q026,Q027,NU_NOTA_MEDIA,TP_PRESENCA,IN_ATEND_ESPECIALIZADO,IN_ATEND_ESPECIFICO
0,180008202043,2018,5300108,Brasília,53,DF,44.0,M,1.0,1,0,NaN,None,NaN,None,2,0,2,0,0,5210208,Iporá,52,GO,NaN,A,A,A,A,1,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,NaN,0,0,0
1,180007197856,2018,2111102,São João dos Patos,21,MA,23.0,F,0.0,3,1,2111102.0,São João dos Patos,21.0,MA,1,5,1,0,0,2111102,São João dos Patos,21,MA,280.0,B,B,A,A,4,B,A,B,C,A,A,B,A,A,A,A,A,A,B,A,A,B,A,A,A,A,A,472.62,1,0,0
2,180008517434,2018,3530607,Mogi das Cruzes,35,SP,23.0,F,0.0,1,1,4209300.0,Lages,42.0,SC,1,3,1,0,0,4314902,Porto Alegre,43,RS,NaN,E,E,C,B,4,C,A,B,C,B,A,B,A,B,A,B,A,A,B,A,A,B,B,B,B,A,A,NaN,0,0,0
3,180007661228,2018,2916401,Itapetinga,29,BA,26.0,F,0.0,3,2,NaN,None,NaN,None,1,8,1,0,0,2916401,Itapetinga,29,BA,NaN,B,C,C,B,4,B,A,B,C,A,A,B,A,A,A,B,A,A,B,B,A,C,A,B,B,A,A,NaN,0,0,0
4,180008787987,2018,2918100,Jeremoabo,29,BA,20.0,M,0.0,0,1,2800308.0,Aracaju,28.0,SE,2,0,2,0,0,2918100,Jeremoabo,29,BA,360.0,C,E,A,D,3,B,A,B,C,A,B,B,A,A,A,A,A,A,B,A,A,B,A,B,B,B,A,NaN,0,0,0


Com relação à **localidade de nascimento**, há duas opções viáveis:

1.   Caso uma parte considerável dos inscritos possuam UF e/ou código de município de nascimento diferente do de residência, talvez possa ser interessante guardar essa informação em uma coluna estilo "TP_MOVIMENTO" e cabe alguma análise de alguma diferença de desempenho ou correlação com algum outro indicador, como renda familiar ou UF de destino, por exemplo.

2.   Caso contrário, considera-se que a localidade permanece a mesma e pode ser desconsiderada qualquer coluna relacionada às localidades de nascimento e residência, tomando como base de geolocalização apenas a UF onde foi realizada a prova.

In [35]:
%%time
# Proporção de pessoas que mudaram de estado
n = len(df[df['SG_UF_NASCIMENTO']!=df['SG_UF_RESIDENCIA']])
print('Pessoas que se mudaram de estado desde que nasceram:\nQuantidade: {}\t{:.2f}%'.format(n, 100*n/len(df)))

Pessoas que se mudaram de estado desde que nasceram:
Quantidade: 793059	14.97%
CPU times: user 7.83 s, sys: 1.21 s, total: 9.04 s
Wall time: 1min 42s


In [36]:
%%time
# Proporção de pessoas que mudaram de município
n = len(df[df['CO_MUNICIPIO_NASCIMENTO']!=df['CO_MUNICIPIO_RESIDENCIA']])
print('Pessoas que se mudaram de cidade desde que nasceram:\nQuantidade: {}\t{:.2f}%'.format(n, 100*n/len(df)))

Pessoas que se mudaram de cidade desde que nasceram:
Quantidade: 2246043	42.41%
CPU times: user 7.62 s, sys: 1.45 s, total: 9.07 s
Wall time: 1min 43s


Os valores nulos serão substituídos pela maior frequência da maior, ou seja, manter-se-á a mesma localidade, mas observando que há uma proporção significativa de inscritos que se mudaram de cidade e Unidade da Federação.

Por conta disso, talvez seja interessante guardar essa informação para eventuais análises.

In [37]:
df['CO_MUNICIPIO_NASCIMENTO'] = df['CO_MUNICIPIO_NASCIMENTO'].fillna(value=df['CO_MUNICIPIO_PROVA'], axis=0)
df['NO_MUNICIPIO_NASCIMENTO'] = df['NO_MUNICIPIO_NASCIMENTO'].fillna(value=df['NO_MUNICIPIO_PROVA'], axis=0)
df['CO_UF_NASCIMENTO'] = df['CO_UF_NASCIMENTO'].fillna(value=df['CO_UF_PROVA'], axis=0)
df['SG_UF_NASCIMENTO'] = df['SG_UF_NASCIMENTO'].fillna(value=df['SG_UF_PROVA'], axis=0)

In [38]:
df['TP_UF_MUDANCA'] = (df['CO_UF_NASCIMENTO']!=df['CO_UF_PROVA']) | (df['CO_UF_RESIDENCIA']!=df['CO_UF_NASCIMENTO'])
df['TP_UF_MUDANCA'] = df['TP_UF_MUDANCA'].astype(int)
df['TP_MUNICIPIO_MUDANCA'] = (df['CO_MUNICIPIO_NASCIMENTO']!=df['CO_MUNICIPIO_PROVA']) | (df['CO_MUNICIPIO_RESIDENCIA']!=df['CO_MUNICIPIO_NASCIMENTO'])
df['TP_MUNICIPIO_MUDANCA'] = df['TP_MUNICIPIO_MUDANCA'].astype(int)

In [39]:
df.head()

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_MUNICIPIO_NASCIMENTO,NO_MUNICIPIO_NASCIMENTO,CO_UF_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,IN_TREINEIRO,IN_SEM_RECURSO,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q026,Q027,NU_NOTA_MEDIA,TP_PRESENCA,IN_ATEND_ESPECIALIZADO,IN_ATEND_ESPECIFICO,TP_UF_MUDANCA,TP_MUNICIPIO_MUDANCA
0,180008202043,2018,5300108,Brasília,53,DF,44.0,M,1.0,1,0,5210208.0,Iporá,52.0,GO,2,0,2,0,0,5210208,Iporá,52,GO,NaN,A,A,A,A,1,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,NaN,0,0,0,1,1
1,180007197856,2018,2111102,São João dos Patos,21,MA,23.0,F,0.0,3,1,2111102.0,São João dos Patos,21.0,MA,1,5,1,0,0,2111102,São João dos Patos,21,MA,280.0,B,B,A,A,4,B,A,B,C,A,A,B,A,A,A,A,A,A,B,A,A,B,A,A,A,A,A,472.62,1,0,0,0,0
2,180008517434,2018,3530607,Mogi das Cruzes,35,SP,23.0,F,0.0,1,1,4209300.0,Lages,42.0,SC,1,3,1,0,0,4314902,Porto Alegre,43,RS,NaN,E,E,C,B,4,C,A,B,C,B,A,B,A,B,A,B,A,A,B,A,A,B,B,B,B,A,A,NaN,0,0,0,1,1
3,180007661228,2018,2916401,Itapetinga,29,BA,26.0,F,0.0,3,2,2916401.0,Itapetinga,29.0,BA,1,8,1,0,0,2916401,Itapetinga,29,BA,NaN,B,C,C,B,4,B,A,B,C,A,A,B,A,A,A,B,A,A,B,B,A,C,A,B,B,A,A,NaN,0,0,0,0,0
4,180008787987,2018,2918100,Jeremoabo,29,BA,20.0,M,0.0,0,1,2800308.0,Aracaju,28.0,SE,2,0,2,0,0,2918100,Jeremoabo,29,BA,360.0,C,E,A,D,3,B,A,B,C,A,B,B,A,A,A,A,A,A,B,A,A,B,A,B,B,B,A,NaN,0,0,0,1,1


#### Colunas "Extras"

Algumas colunas foram formatadas pelo Inep de forma a representar dados não informados com "1". Por isso, muitas não são detectadas com *isna()* e é preciso abordá-las individualmente de acordo com o Dicionário dos Microdados fornecido também pelo Inep.

Entre as que ainda estão presentes no Dataframe, constituem esse grupo:

*   TP_COR_RACA
*   TP_NACIONALIDADE
*   TP_ANO_CONCLUIU
*   TP_ESCOLA

Sendo assim, a começar por **TP_COR_RACA**, realiza-se uma análise de cada uma dessas colunas para ser decidido como serão tratados os dados que os inscritos escolheram não responder. Para isso, tenha-se que "0 = Não declarado"

In [40]:
df['TP_COR_RACA'].value_counts().compute()

3    2462613
1    1911846
2     671147
4     118362
0      99689
5      32355
Name: TP_COR_RACA, dtype: int64

É possível observar que aqueles que escolheram não declarar ou não se sentiram representados pelas outras alternativas não representam, necessariamente, um grupo de dados faltantes. É possível tomar como um grupo como os outros.

A coluna deve, então, permanecer da mesma forma.

Já com  **TP_NACIONALIDADE**, dado que "0 = Não informado", tem-se:


In [41]:
df['TP_NACIONALIDADE'].value_counts().compute()

1    5118253
2     158004
4      11973
3       6481
0       1301
Name: TP_NACIONALIDADE, dtype: int64

A proporção daqueles que não informaram não representam relevância perante os outros dados. Como a grande maioria é "1 = Brasileiro(a)", substituir-se-á todos zeros por um.

In [42]:
df['TP_NACIONALIDADE'] = df['TP_NACIONALIDADE'].replace(0,1)

Para a **coluna TP_ANO_CONCLUIU**, com "0 = Não informado", tem-se:

In [43]:
df['TP_ANO_CONCLUIU'].value_counts().compute()

0     2285989
1      680981
12     480968
2      444546
3      322103
4      245985
5      190867
6      158989
7      123415
8      112711
9       96718
10      79609
11      73131
Name: TP_ANO_CONCLUIU, dtype: int64

Aqueles que não concluíram o Ensino Médio ainda também marcaram "0" nesse item do formulário. Apesar de haver alguns inscritos que marcaram desta forma mesmo assinalando no item anterior (TP_ST_CONCLUSAO) que já haviam concluído o Ensino Médio, viu desnecessário distinguí-los, visto que representam uma quantidade pequena com relação ao todo.

Além disso, aproveita-se para fazer a seguinte alocação:

*   Serão agrupados todos aqueles que concluíram o Ensino Médio a 10 (dez) anos ou mais antes de realizarem o respectivo Exame, ou seja, os valores da coluna TP_ANO_CONCLUIU maiores que 10 serão substituídos por 10.

Desta forma, obtem-se uma clusterização mais "arredondada" mas que mantém a informação relevante.

In [44]:
condition = df['TP_ANO_CONCLUIU'] > 10
df['TP_ANO_CONCLUIU'] = df['TP_ANO_CONCLUIU'].mask(condition, 10)

Com relação à **coluna TP_ESCOLA**, sendo "1 = Não Respondeu": 



In [45]:
df['TP_ESCOLA'].value_counts().compute()

1    3930551
2    1106812
4     245194
3      13455
Name: TP_ESCOLA, dtype: int64

Devido ao número muito grande de inscritos que não responderam, será mantida esta coluna de forma que o grupo que respondeu possa ser agrupado na etapa de Análise Exploratória.

In [46]:
df.head()

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_MUNICIPIO_NASCIMENTO,NO_MUNICIPIO_NASCIMENTO,CO_UF_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,IN_TREINEIRO,IN_SEM_RECURSO,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q026,Q027,NU_NOTA_MEDIA,TP_PRESENCA,IN_ATEND_ESPECIALIZADO,IN_ATEND_ESPECIFICO,TP_UF_MUDANCA,TP_MUNICIPIO_MUDANCA
0,180008202043,2018,5300108,Brasília,53,DF,44.0,M,1.0,1,1,5210208.0,Iporá,52.0,GO,2,0,2,0,0,5210208,Iporá,52,GO,NaN,A,A,A,A,1,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,NaN,0,0,0,1,1
1,180007197856,2018,2111102,São João dos Patos,21,MA,23.0,F,0.0,3,1,2111102.0,São João dos Patos,21.0,MA,1,5,1,0,0,2111102,São João dos Patos,21,MA,280.0,B,B,A,A,4,B,A,B,C,A,A,B,A,A,A,A,A,A,B,A,A,B,A,A,A,A,A,472.62,1,0,0,0,0
2,180008517434,2018,3530607,Mogi das Cruzes,35,SP,23.0,F,0.0,1,1,4209300.0,Lages,42.0,SC,1,3,1,0,0,4314902,Porto Alegre,43,RS,NaN,E,E,C,B,4,C,A,B,C,B,A,B,A,B,A,B,A,A,B,A,A,B,B,B,B,A,A,NaN,0,0,0,1,1
3,180007661228,2018,2916401,Itapetinga,29,BA,26.0,F,0.0,3,2,2916401.0,Itapetinga,29.0,BA,1,8,1,0,0,2916401,Itapetinga,29,BA,NaN,B,C,C,B,4,B,A,B,C,A,A,B,A,A,A,B,A,A,B,B,A,C,A,B,B,A,A,NaN,0,0,0,0,0
4,180008787987,2018,2918100,Jeremoabo,29,BA,20.0,M,0.0,0,1,2800308.0,Aracaju,28.0,SE,2,0,2,0,0,2918100,Jeremoabo,29,BA,360.0,C,E,A,D,3,B,A,B,C,A,B,B,A,A,A,A,A,A,B,A,A,B,A,B,B,B,A,NaN,0,0,0,1,1


Por fim, redefinindo os tipos das colunas 

In [47]:
dtype = {'NU_IDADE': int,
         'TP_ESTADO_CIVIL': int,
         'CO_MUNICIPIO_NASCIMENTO': int,
         'CO_UF_NASCIMENTO': int}
df = df.astype(dtype)

### Salvando Arquivo

Salvando arquivo no formato .parquet para a fase de Análise Exploratória dos Dados e para a preparação dos dados a fim de servirem de entrada aos algoritmos de Machine Learning.

In [48]:
save_path = f'/content/drive/MyDrive/Colab Notebooks/IC_Data_Science_ENEM/Dados/PRE_PROCESSED_ENEM_{str(ano)}.pq'
df.to_parquet(save_path, write_index=None)